#### Each sample in the train and test set has the following information:

    - The text of a tweet
    - A keyword from that tweet (although this may be blank!)
    - The location the tweet was sent from (may also be blank)

**We are predicting whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.**


In [1]:
from IPython.core.debugger import set_trace

%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="seaborn")
%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
!cd data && ls

sample_submission.csv test.csv              train.csv


<IPython.core.display.Javascript object>

In [3]:
train = pd.read_csv(f"data/train.csv")
test = pd.read_csv(f"data/test.csv")

<IPython.core.display.Javascript object>

In [4]:
train.head().T

,0,1,2,3,4
id,1,4,5,6,7
keyword,NaN,NaN,NaN,NaN,NaN
location,NaN,NaN,NaN,NaN,NaN
text,Our Deeds are the Reason of this #earthquake M...,Forest fire near La Ronge Sask. Canada,All residents asked to 'shelter in place' are ...,"13,000 people receive #wildfires evacuation or...",Just got sent this photo from Ruby #Alaska as ...
target,1,1,1,1,1


<IPython.core.display.Javascript object>

In [5]:
test.head().T

,0,1,2,3,4
id,0,2,3,9,11
keyword,NaN,NaN,NaN,NaN,NaN
location,NaN,NaN,NaN,NaN,NaN
text,Just happened a terrible car crash,"Heard about #earthquake is different cities, s...","there is a forest fire at spot pond, geese are...",Apocalypse lighting. #Spokane #wildfires,Typhoon Soudelor kills 28 in China and Taiwan


<IPython.core.display.Javascript object>

### Clean Data
**Remove URLs and HTML**

In [6]:
import re


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

<IPython.core.display.Javascript object>

In [7]:
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

<IPython.core.display.Javascript object>

**Remove punctuation**

In [8]:
import string


def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

<IPython.core.display.Javascript object>

In [9]:
train["text"] = train.text.map(lambda x: remove_URL(x))
train["text"] = train.text.map(lambda x: remove_html(x))
train["text"] = train.text.map(lambda x: remove_emoji(x))
train["text"] = train.text.map(lambda x: remove_punct(x))

<IPython.core.display.Javascript object>

**Remove stopwords**

In [10]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

<IPython.core.display.Javascript object>

In [11]:
train["text"] = train["text"].map(remove_stopwords)

<IPython.core.display.Javascript object>

In [12]:
train.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

<IPython.core.display.Javascript object>

### Basic NLP

In [13]:
from collections import Counter

# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

<IPython.core.display.Javascript object>

In [14]:
text = train.text

counter = counter_word(text)

<IPython.core.display.Javascript object>

In [15]:
len(counter)

17971

<IPython.core.display.Javascript object>

In [16]:
counter

Counter({'deeds': 2,
         'reason': 20,
         'earthquake': 50,
         'may': 88,
         'allah': 9,
         'forgive': 2,
         'us': 164,
         'forest': 65,
         'fire': 250,
         'near': 54,
         'la': 25,
         'ronge': 1,
         'sask': 1,
         'canada': 11,
         'residents': 8,
         'asked': 9,
         'shelter': 6,
         'place': 26,
         'notified': 1,
         'officers': 8,
         'evacuation': 50,
         'orders': 11,
         'expected': 15,
         '13000': 4,
         'people': 196,
         'receive': 2,
         'wildfires': 11,
         'california': 117,
         'got': 112,
         'sent': 13,
         'photo': 41,
         'ruby': 1,
         'alaska': 6,
         'smoke': 48,
         'pours': 1,
         'school': 66,
         'rockyfire': 4,
         'update': 37,
         'hwy': 9,
         '20': 26,
         'closed': 20,
         'directions': 1,
         'due': 31,
         'lake': 14,
         'co

<IPython.core.display.Javascript object>

In [17]:
num_words = len(counter)

# Max number of words in a sequence
max_length = 20

<IPython.core.display.Javascript object>

Train / test split

In [18]:
train_size = int(train.shape[0] * 0.8)

train_sentences = train.text[:train_size]
train_labels = train.target[:train_size]

test_sentences = train.text[train_size:]
test_labels = train.target[train_size:]

<IPython.core.display.Javascript object>

### we can use keras to make Tokenizer

In [19]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [20]:
word_index = tokenizer.word_index

<IPython.core.display.Javascript object>

In [21]:
word_index

{'like': 1,
 'amp': 2,
 'fire': 3,
 'im': 4,
 'get': 5,
 'via': 6,
 'new': 7,
 'people': 8,
 'news': 9,
 'dont': 10,
 'emergency': 11,
 'one': 12,
 '2': 13,
 'us': 14,
 'video': 15,
 'disaster': 16,
 'burning': 17,
 'body': 18,
 'would': 19,
 'buildings': 20,
 'police': 21,
 'crash': 22,
 'first': 23,
 'california': 24,
 'still': 25,
 'man': 26,
 'got': 27,
 'know': 28,
 'day': 29,
 'back': 30,
 'going': 31,
 'two': 32,
 'time': 33,
 'full': 34,
 'accident': 35,
 'see': 36,
 'world': 37,
 'attack': 38,
 'nuclear': 39,
 'youtube': 40,
 'may': 41,
 'love': 42,
 'go': 43,
 'rt': 44,
 'many': 45,
 'cant': 46,
 '3': 47,
 'watch': 48,
 'collapse': 49,
 'dead': 50,
 'today': 51,
 'car': 52,
 'mass': 53,
 'want': 54,
 'years': 55,
 'work': 56,
 'train': 57,
 'last': 58,
 'good': 59,
 'think': 60,
 'families': 61,
 'hiroshima': 62,
 'life': 63,
 'fires': 64,
 'best': 65,
 'could': 66,
 'say': 67,
 'u': 68,
 'death': 69,
 'hot': 70,
 'forest': 71,
 'way': 72,
 'killed': 73,
 'need': 74,
 'legion

<IPython.core.display.Javascript object>

In [22]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

<IPython.core.display.Javascript object>

In [23]:
train_sequences[0]

[3739, 696, 235, 41, 1282, 3740, 14]

<IPython.core.display.Javascript object>

### Calculate the pad_sequences

In [24]:
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

<IPython.core.display.Javascript object>

In [25]:
train_padded[0]

array([3739,  696,  235,   41, 1282, 3740,   14,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

<IPython.core.display.Javascript object>

In [26]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

<IPython.core.display.Javascript object>

In [27]:
print(train.text[0])
print(train_sequences[0])

deeds reason earthquake may allah forgive us
[3739, 696, 235, 41, 1282, 3740, 14]


<IPython.core.display.Javascript object>

**Check inverse**

In [28]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

<IPython.core.display.Javascript object>

In [29]:
def decode(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

<IPython.core.display.Javascript object>

In [30]:
decode(train_sequences[0])

'deeds reason earthquake may allah forgive us'

<IPython.core.display.Javascript object>

In [31]:
print(f"Shape of train {train_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (6090, 20)
Shape of test (1523, 20)


<IPython.core.display.Javascript object>

###  Cleck the models

In [32]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

model = Sequential()

model.add(Embedding(num_words, 32, input_length=max_length))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))


optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

<IPython.core.display.Javascript object>

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            575072    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 599,969
Trainable params: 599,969
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [34]:
history = model.fit(
    train_padded, train_labels, epochs=20, validation_data=(test_padded, test_labels),
)

/Users/radufotolescu/.local/share/virtualenvs/useful-jAUW_uHU/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/20
6090/6090 [==============================] - 3s 504us/step - loss: 0.6532 - accuracy: 0.6069 - val_loss: 0.5211 - val_accuracy: 0.7557
Epoch 2/20
6090/6090 [==============================] - 2s 383us/step - loss: 0.3937 - accuracy: 0.8332 - val_loss: 0.5107 - val_accuracy: 0.7708
Epoch 3/20
6090/6090 [==============================] - 2s 393us/step - loss: 0.2408 - accuracy: 0.9102 - val_loss: 0.5752 - val_accuracy: 0.7708
Epoch 4/20
6090/6090 [==============================] - 2s 397us/step - loss: 0.1609 - accuracy: 0.9481 - val_loss: 0.6074 - val_accuracy: 0.7564
Epoch 5/20
6090/6090 [==============================] - 2s 399us/step - loss: 0.1106 - accuracy: 0.9639 - val_loss: 0.7123 - val_accuracy: 0.7531
Epoch 6/20
6090/6090 [==============================] - 2s 386us/step - loss: 0.0878 - accuracy: 0.9732 - val_loss: 0.7751 - val_accuracy: 0.7459
Epoch 7/20
6090/6090 [==============================] - 2s 388us/step - loss

<IPython.core.display.Javascript object>

### End